In [35]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy

#LDPC codes

##1.

Write a function that receives a parity check matrix H and builds a systematic encoding matrix G for it. The function should return two matrices: ˆH and G, such that ˆH is equal to H up to a column permutation and ˆHGt = 0 for all t.

In [208]:
#initialize H
H = np.array([[1, 1, 1, 1, 0, 0],[0, 0, 1, 1, 0, 1],[1, 0, 0, 1, 1, 0]], dtype=int)

print(H)


[[1 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 0 0 1 1 0]]


In [87]:
def rref(H):

    ''' return reduce row echelon form'''
    m, n = H.shape
    # k = n-m

    i = 0
    j = 0
    r = 0

    H_hat = H.copy()

    while True:

        if i >= m or j >= n:
            break
        
        # print(H_hat)
        
        #check if entry = 0
        if H_hat[i,j]== 0:

            #find next row with non zero entry to swap
            r = i

            while r < m and H_hat[r,j] == 0 :
                r += 1
            
            #if no nonzero found skip to next column
            if r == m:
                j+=1
                break
        
        #swap row
        zero_row = H_hat[i,:].copy()
        nonzero = H_hat[r,:].copy()
        H_hat[i,:] = nonzero
        H_hat[r,:] = zero_row
        
        # print(H_hat)

        #perform row elimination
        for row in range(m):
            if row == i:
                continue
            #if entry does not = 0, add nonzero row, remember this is in F2 (so mod 2 addition)
            if H_hat[row,j] != 0:
                H_hat[row,:] = (H_hat[row,:] + H_hat[i,:]) % 2
        
        # print(H_hat)

        i+=1
        j+=1
    
    return H_hat


In [139]:
def gen_G(H):
    m, n = H.shape
    k = n-m

    H_hat = rref(H)
    # print("H_hat:\n",H_hat)
    P = H_hat[:, m:n]
    # print("P:\n",P)
    G = np.vstack((P, np.identity(k)))
    # print("G\n",G) 

    return H_hat, G
    

In [141]:
H_hat, G = gen_G(H)

print("H_hat:\n",H_hat, "\nG:\n", G)

H_hat:
 [[1 0 0 1 1 0]
 [0 1 0 1 1 1]
 [0 0 1 1 0 1]] 
G:
 [[1. 1. 0.]
 [1. 1. 1.]
 [1. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


##3.

Write an LDPC-decoder based on Loopy Belief Propagation for Binary Symmetric Channel.

In [457]:
#import y1
file_y = open("y1.txt")
y_content = file_y.read()
file_y.close()
y = y_content.split("\n")
# print(y1)
y = y[:-1]
y1 = np.array([int(i) for i in y])

y1.shape

(1000,)

In [456]:
#import H1
file_h = open("H1.txt")
H_content = file_h.read()
H1 = H_content.split("\n")
H1 = H1[:-1]
H = [np.array(i.split(), dtype=int) for i in H1]
h1 = np.vstack(H)

h1.shape


(750, 1000)

In [466]:
def decode(H, y, p=.1, max_iter = 20):

    success = -1
    m, n = H.shape

    #list of factor checks
    B = []
    for i in range(m):
        B.append(np.where(H[i]==1)[0])
    # print("Bit Check:",B)

    #probability y given x

    #if true val is 1
    P_y_x1 = np.zeros(len(y))
    #if true val is 0
    P_y_x0 = np.zeros(len(y))

    for i in range(len(y)):
        P_y_x1[i] = p**((y[i]+1)%2) * (1-p)**((y[i]+1+1)%2)
        P_y_x0[i] = p**(y[i]) * (1-p)**((y[i]+1)%2)

    #initialize
    M = np.zeros((m,n))

    # print(M)

    for i in range(m):
        for j in range(n):
            #for efficiency: take difference and use log liklihood:
            # then u(0) - u(1) -> log(u(0))-log(u(1)) = log(u(0)/u(1))
            M[i,j] = np.log([P_y_x0[j]/P_y_x1[j]])
    
    # print("Initial Message:\n",M)
   

    for iter in range(max_iter):
        
        #factor to variable 
        M_f = np.zeros((m,n))

        for j in range(len(B)):
                
            new_row = np.zeros(n)
            # print("\nrow:", j)

            #get values of M for all indexes where h==1 for the jth row
            prod = M[j,(B[j])]
            # print(prod)
            
            for i in range(len(prod)):

                # print("\nindex:", B[j][i])

                #take product of all incoming messages excludng the ith bit
                to_prod = np.hstack((prod[0:i],prod[i+1:len(prod)]))
                product = np.prod(np.tanh(to_prod/2))
                to_log = (1+product)/(1-product)
                # print("log:",to_log)
                
                new_row[B[j][i]] = np.log(to_log)
            
            # print(j, new_row)
                
            M_f[j,:] = new_row
            # M[j,:] = new_row
        
        # print(M_f)

        #calc log liklihood of posterior
        posterior = np.sum(M_f, axis = 0)
        
        for i in range(n):
            # calc log like
            posterior[i] += np.log([P_y_x0[i]/P_y_x1[i]])
        
        # print("\nposterior",posterior)
        
        #update prediction
        z = np.zeros(n)
        for i in range(n):
            if posterior[i]>0:
                z[i]=0
            else:
                z[i]=1
        
        # print("\nprediction:", z)
            
        #test, dont forget mod 2
        test = np.matmul(H, z.T)% 2

        #check if parity conditions met or reached max iterations
        if iter == max_iter or np.all(test == 0):
            success = 0
            break
        
        #if not continue
        else:
            
            #update variable to factor
            for i in range(n):
                for j in range(m):
                    M[j,i] = posterior[i] - M_f[j,i]
    
    return success, iter+1, z


In [465]:
H_test = np.array([[1, 1, 0, 1, 0, 0],[0, 1, 1, 0, 1, 0],[1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1]], dtype=int)
y_test = np.array([1,0,1,0,1,1])

# print(H_test)

decode(H_test,y_test, p=.1, max_iter=20)

# np.array(np.where(H_test[1] == 1)[0])

Bit Check: [array([0, 1, 3]), array([1, 2, 4]), array([0, 4, 5]), array([2, 3, 5])]
Initial Message:
 [[-2.19722458  2.19722458 -2.19722458  2.19722458 -2.19722458 -2.19722458]
 [-2.19722458  2.19722458 -2.19722458  2.19722458 -2.19722458 -2.19722458]
 [-2.19722458  2.19722458 -2.19722458  2.19722458 -2.19722458 -2.19722458]
 [-2.19722458  2.19722458 -2.19722458  2.19722458 -2.19722458 -2.19722458]]


(0, 1, array([0., 0., 1., 0., 1., 1.]))

In [468]:
success,iter,decoded = decode(h1,y1, p=.1, max_iter=20)

In [470]:
print(success, iter, "\n",decoded)

0 8 
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1.
 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.

In [506]:
#translate 
decoded.shape

x = np.reshape(decoded.astype(int), (125,8))
x = x.astype(str)
lst = x.tolist()

b = []
for i in range(len(lst)):
    b.append(''.join(map(str, lst[i])))

for i in range(31):
    print(chr(int(b[i],2)))


H
a
p
p
y
 
H
o
l
i
d
a
y
s
!
 
D
m
i
t
r
y
&
D
a
v
i
d
 
:
)


In [505]:
len(x)

125